<h1 align=center><font size = 5>Week 3 Part1</font></h1>

## Week 3 - Start of Part 1

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [4]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [5]:
import bs4 as bs
import urllib.request
print('More Libraries imported.')

More Libraries imported.


## Load in the file using BeautifulSoup 4

#### Tranform the data into a *pandas* dataframe

In [6]:
#source = urllib.request.urlopen('https://pythonprogramming.net/parsememcparseface/').read()
source = urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').read()
soup = bs.BeautifulSoup(source,'lxml')

table = soup.table
table = soup.find('table')
table_rows = table.find_all('tr')
#print(table_rows)


df = pd.DataFrame()
header_list = []
rows_list = []


for hd in table_rows:
    th = hd.find_all('th')
    row = [i.text for i in th]
    #print("header",row)
    header_list.append(row)
    #df.append(row)
    
for tr in table_rows:
    td = tr.find_all('td')
    row = [i.text for i in td]
    #print(row)
    rows_list.append(row)
    #df.append(row)
    
#print(rows_list)
df = pd.DataFrame(rows_list)
print(df.head())

      0             1                   2
0  None          None                None
1   M1A  Not assigned      Not assigned\n
2   M2A  Not assigned      Not assigned\n
3   M3A    North York         Parkwoods\n
4   M4A    North York  Victoria Village\n


## testing to see how to load in using Pandas (** but not used **)

In [7]:
# I can also read it in useing Pandas,   but when doing this way I get another tables 
target = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#dfs = pd.read_html('https://pythonprogramming.net/parsememcparseface/',header=0)
dfs = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M',header=0)
for df_test in dfs:
    print(df_test.head(2))

  Postcode       Borough Neighbourhood
0      M1A  Not assigned  Not assigned
1      M2A  Not assigned  Not assigned
                                          Unnamed: 0  \
0  NL NS PE NB QC ON MB SK AB BC NU/NT YT A B C E...   
1                                                 NL   

                               Canadian postal codes  \
0  NL NS PE NB QC ON MB SK AB BC NU/NT YT A B C E...   
1                                                 NS   

                                          Unnamed: 2 Unnamed: 3 Unnamed: 4  \
0  NL NS PE NB QC ON MB SK AB BC NU/NT YT A B C E...        NaN        NaN   
1                                                 PE         NB         QC   

  Unnamed: 5 Unnamed: 6 Unnamed: 7 Unnamed: 8 Unnamed: 9 Unnamed: 10  \
0        NaN        NaN        NaN        NaN        NaN         NaN   
1         QC         QC         ON         ON         ON          ON   

  Unnamed: 11 Unnamed: 12 Unnamed: 13 Unnamed: 14 Unnamed: 15 Unnamed: 16  \
0         NaN   

<a id='item1'></a>

### Put Columns on the dataframe

In [8]:
# define the dataframe columns
postal_table = pd.DataFrame()
#neighborhoods = df

column_names = ["PostalCode",'Borough', 'Neighborhood'] # 'Latitude', 'Longitude'] 
postal_table = pd.DataFrame(df.values,columns=column_names)

print(postal_table.head(5))
postal_table = postal_table.drop(postal_table.index[0])
print(postal_table.head(5))



  PostalCode       Borough        Neighborhood
0       None          None                None
1        M1A  Not assigned      Not assigned\n
2        M2A  Not assigned      Not assigned\n
3        M3A    North York         Parkwoods\n
4        M4A    North York  Victoria Village\n
  PostalCode           Borough        Neighborhood
1        M1A      Not assigned      Not assigned\n
2        M2A      Not assigned      Not assigned\n
3        M3A        North York         Parkwoods\n
4        M4A        North York  Victoria Village\n
5        M5A  Downtown Toronto      Harbourfront\n


### Remove "not assigned" and clean up the "/n"

In [9]:
#dropping rows where Borough is 'Not assigned'
postal_table.drop(postal_table[postal_table.Borough=='Not assigned'].index,inplace=True)
postal_table.index=range(len(postal_table.index))
postal_table = postal_table.replace('\n','', regex=True)
print(postal_table.head(10))
print(postal_table.shape)

  PostalCode           Borough      Neighborhood
0        M3A        North York         Parkwoods
1        M4A        North York  Victoria Village
2        M5A  Downtown Toronto      Harbourfront
3        M5A  Downtown Toronto       Regent Park
4        M6A        North York  Lawrence Heights
5        M6A        North York    Lawrence Manor
6        M7A      Queen's Park      Not assigned
7        M9A         Etobicoke  Islington Avenue
8        M1B       Scarborough             Rouge
9        M1B       Scarborough           Malvern
(211, 3)


## replace the "NOT Assigned" Neighbourhoods with the Borough name

In [10]:
#replace the "NOT Assigned" Neighbourhoods with the Borough name

#np.where(condition, then, else)
postal_table['Neighborhood']=np.where(postal_table['Neighborhood']=='Not assigned',postal_table['Borough'],postal_table['Neighborhood'])

#print(len(postal_table))
#print(postal_table['Neighborhood'])
#for i in range(0,len(postal_table)):
#    if postal_table['Neighborhood']=='Not assigned':
#        postal_table['Neighborhood'] = postal_table['Borough']

postal_table.head(10)


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [11]:
#combine all of the duplicated postcodes and combine neighbourhoods that are in the same borough
postal_table1=postal_table.groupby(by=['PostalCode','Borough']).agg(lambda x: ','.join(x))
postal_table1.reset_index(level=['PostalCode','Borough'],inplace=True)
postal_table1

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [12]:
postal_table1.shape

(103, 3)

## Week 3 - End of Part 1
---
## Week 3 - Start of Part 2

## use the file they gave us and load it in 

In [13]:
toronto_data = pd.read_csv('http://cocl.us/Geospatial_data')
toronto_data.rename(columns={'Postal Code':'PostalCode'},inplace=True)
print(toronto_data.shape)
toronto_data.head(10)

(103, 3)


,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


### in this section we will put the two dataframes together

In [14]:
toronto_merged = pd.merge(postal_table1,toronto_data,on='PostalCode')
toronto_merged

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


In [15]:
toronto_merged.shape

(103, 5)

## Week 3 - End of Part 2
---
## Week 3 - Start of Part 3

In [16]:
from geopy.geocoders import Nominatim
!conda install -c conda-forge folium=0.5.0
import folium
print("done")

Solving environment: | 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/linux-64::anaconda==5.3.1=py37_0
  - defaults/linux-64::astropy==3.0.4=py37h14c3975_0
  - defaults/linux-64::bkcharts==0.2=py37_0
  - defaults/linux-64::blaze==0.11.3=py37_0
  - defaults/linux-64::bokeh==0.13.0=py37_0
  - defaults/linux-64::bottleneck==1.2.1=py37h035aef0_1
  - defaults/linux-64::dask==0.19.1=py37_0
  - defaults/linux-64::datashape==0.5.4=py37_1
  - defaults/linux-64::mkl-service==1.1.2=py37h90e4bf4_5
  - defaults/linux-64::numba==0.39.0=py37h04863e7_0
  - defaults/linux-64::numexpr==2.6.8=py37hd89afb7_0
  - defaults/linux-64::odo==0.5.1=py37_0
  - defaults/linux-64::pytables==3.4.4=py37ha205bf6_0
  - defaults/linux-64::pytest-arraydiff==0.2=py37h39e3cac_0
  - defaults/linux-64::pytest-astropy==0.4.0=py37_0
  - defaults/linux-64::pytest-doctestplus==0.1.3=py37_0
  - defaults/linux-64::pywavelets==1.0.0=py37h

In [17]:
#get geo-graphical date coordinates for Toronto
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("done", address)
print('The geograpical coordinates for {} are {}, {}'.format(address,latitude, longitude))

done Toronto, Ontario
The geograpical coordinates for Toronto, Ontario are 43.653963, -79.387207


## I will be looking at Boroughs that have the word "Toronto"

In [18]:
#select only boroughs that contain the word Toronto
neighborhoods=toronto_merged[toronto_merged['Borough'].str.contains("Toronto")]
neighborhoods

,PostalCode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
45,M4P,Central Toronto,Davisville North,43.712751,-79.390197
46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
47,M4S,Central Toronto,Davisville,43.704324,-79.388790
48,M4T,Central Toronto,"Moore Park,Summerhill East",43.689574,-79.383160
49,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",43.686412,-79.400049


In [25]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
#map_toronto

Then let's loop through the data and fill the dataframe one row at a time.

In [38]:
# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)

Quickly examine the resulting dataframe.

In [39]:
map_toronto